In [30]:
import warnings
import numpy as np
import pandas as pd
import copy
from pathlib import Path
import os

import torch
import torch.nn as nn

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

import pytorch_forecasting
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss

import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile


device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [31]:
def zdorovaya_dura(
    data: pd.DataFrame,
    batch_size: int,
    target: str,
    index_col: str,
    max_prediction_length: int,
    time_varying_unknown_reals: list,
    group_by: list,
    epochs: int):
    
    training = TimeSeriesDataSet(
        data=data, 
        time_idx=index_col,
        target=target,
        group_ids=group_by,
        add_relative_time_idx=True,
        time_varying_unknown_reals=time_varying_unknown_reals,
        max_prediction_length=max_prediction_length,
        min_encoder_length=1,
        max_encoder_length=max_prediction_length,
#         allow_missing_timesteps=True
    )
    
    validation = TimeSeriesDataSet.from_dataset(training, data
                                                , predict=True, stop_randomization=True)
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10
                                              , num_workers=0)
    
    lr_logger = LearningRateMonitor()
    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4
                                        , patience=10, verbose=False, mode="min")
    logger = TensorBoardLogger("lightning_logs")

    trainer = pl.Trainer(
        max_epochs=epochs,
        gpus=1,
        enable_model_summary=True,
        gradient_clip_val=0.1,
        limit_train_batches=30,
        callbacks=[lr_logger, early_stop_callback],
        logger=logger,
    )

    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.01,
        hidden_size=16,
        attention_head_size=1,
        dropout=0.1,
        hidden_continuous_size=8,
        loss=QuantileLoss(),
        log_interval=25,
        output_size=7,
        reduce_on_plateau_patience=4,
    )
    
    trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    )
    
    return tft, tft.predict(data)

In [40]:
data_for_forecasting_0_prepared = pd.read_csv(r"C:\Users\sahab\Desktop\slovarik\spb_prepared_live_ap_0.csv")
data_for_forecasting_1_prepared = pd.read_csv(r"C:\Users\sahab\Desktop\slovarik\spb_prepared_live_ap_1.csv")
data_for_forecasting_2_prepared = pd.read_csv(r"C:\Users\sahab\Desktop\slovarik\spb_prepared_live_ap_2.csv")
data_for_forecasting_3_prepared = pd.read_csv(r"C:\Users\sahab\Desktop\slovarik\spb_prepared_live_ap_3.csv")
data_for_forecasting_4_prepared = pd.read_csv(r"C:\Users\sahab\Desktop\slovarik\spb_prepared_live_ap_4.csv")

In [41]:
data_for_forecasting_4_prepared

,Unnamed: 0,date,price
0,0,2022-08-29,2.100068e+07
1,1,2022-08-30,2.705992e+07
2,2,2022-08-31,9.200000e+06
3,3,2022-09-01,2.804097e+07
4,4,2022-09-02,2.037549e+07
...,...,...,...
361,361,2023-08-25,2.297214e+07
362,362,2023-08-26,1.283333e+07
363,363,2023-08-27,1.411800e+07
364,364,2023-08-28,2.557556e+07


In [42]:
data_for_forecasting_0_prepared["class"] = 0
data_for_forecasting_1_prepared["class"] = 1
data_for_forecasting_2_prepared["class"] = 2
data_for_forecasting_3_prepared["class"] = 3
data_for_forecasting_4_prepared["class"] = 4

data_for_forecasting_0_prepared["time_index"] = np.arange(366)
data_for_forecasting_1_prepared["time_index"] = np.arange(366)
data_for_forecasting_2_prepared["time_index"] = np.arange(366)
data_for_forecasting_3_prepared["time_index"] = np.arange(366)
data_for_forecasting_4_prepared["time_index"] = np.arange(366)

In [43]:
tft_df = pd.concat(
    [
        data_for_forecasting_0_prepared,
        data_for_forecasting_1_prepared,
        data_for_forecasting_2_prepared,
        data_for_forecasting_3_prepared,
        data_for_forecasting_4_prepared
    ], ignore_index=True
)

In [45]:
tft_df = tft_df.drop(columns="Unnamed: 0")

In [49]:
tft, answer = zdorovaya_dura(
    data=tft_df,
    batch_size=14,
    target="price",
    index_col="time_index",
    time_varying_unknown_reals=["price"],
    group_by=["class"],
    epochs=100,
    max_prediction_length=90
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 0     
3  | prescalers                         | ModuleDict                      | 48    
4  | static_variable_selection          | VariableSelectionNetwork        | 528   
5  | encoder_variable_selection         | VariableSelectionNetwork        | 1.2 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 528   
7  | static_context_variable_selection  | GatedResid

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [51]:
time_index = pd.date_range(start='29/8/2023', periods=90)

answer_0_df = pd.Series(answer[0], index=time_index)
answer_1_df = pd.Series(answer[1], index=time_index)
answer_2_df = pd.Series(answer[2], index=time_index)
answer_3_df = pd.Series(answer[3], index=time_index)
answer_4_df = pd.Series(answer[4], index=time_index)
